In [1]:
spark 

In [5]:
# 讀入transaction.csv檔案後存入變數transactions 
# format("csv")=>設定匯入為csv檔
# option("header","true")=>保留表頭
transactions = spark.read.format("csv").option("header", "true").load("/home/cloudera/KKBOX/transactions.csv") 

In [6]:
# 查看transactions.csv裡有幾筆紀錄 ? 總共有21,547,746筆紀錄 => KKBOX會員總數為六百多萬,由此推斷這裡面有一個會員擁有多筆紀錄的情況 !  
transactions.count() 

21547746

In [7]:
# 查看transactions.csv的欄位內容為何 ?
transactions.columns  

['msno',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'transaction_date',
 'membership_expire_date',
 'is_cancel']

In [2]:
# 讀入data.csv檔案後存入變數data
# 設定為csv格式檔 
# 表格裡的每個的欄位以"|"分隔 
data = spark.read.format("csv").option("sep","|").load("/home/cloudera/KKBOX/data.csv") 

In [8]:
# 確認表格data.csv的資料筆數  
data.count() 

379478

In [15]:
# 查看data.csv的前5筆資料 
data.show(5) 

+---+--------------------+---+---+------+--------+---+
|_c0|                 _c1|_c2|_c3|   _c4|     _c5|_c6|
+---+--------------------+---+---+------+--------+---+
|  0|+/namlXq+u3izRjHC...| 31| 15|  male|20060603|  9|
|  0|+0/X9tkmyHyet9X80...| 31|  9|  male|20040330|  9|
|  0|+09YGn842g6h2EZUX...| 29| 15|  male|20080322|  9|
|  1|+0RJtbyhoPAHPa+34...| 29| 13|female|20120612|  3|
|  0|+0jTOa6KGPk1vtNTw...| 24|  5|female|20140320|  9|
+---+--------------------+---+---+------+--------+---+
only showing top 5 rows



In [16]:
# 更改data.csv的欄位名稱: "_c1" => "ID" 
data.selectExpr("_c1 as ID").show(5) 

+--------------------+
|                  ID|
+--------------------+
|+/namlXq+u3izRjHC...|
|+0/X9tkmyHyet9X80...|
|+09YGn842g6h2EZUX...|
|+0RJtbyhoPAHPa+34...|
|+0jTOa6KGPk1vtNTw...|
+--------------------+
only showing top 5 rows



In [17]:
# 更改過欄位名稱的表格存入變數data01 
data01 = data.selectExpr("_c1 as ID") 

In [18]:
# 將transactions.csv併入表格data01(以會員ID為合併基準)存入變數joined_df00 => 保留表格data01裡所有的會員資料
# 過濾transactions.csv中不存在於表格data01裡的會員記錄 
joined_df00 = transactions.join(data01,\
transactions.msno == data01.ID,"right_outer") 

In [19]:
# 清查合併之後的表格joined_df00有幾筆資料 ? 
# 379,478 < 合理的記錄筆數 < 21,547,746筆 => 6,645,531在合理的範圍 
joined_df00.count() 

6645431

In [20]:
# 將joined_df00裡的資料匯出: 
# coalesce(1)=> Spark匯出的一個大檔案檔案預設分割成若干個小檔案以儲存,使用這個函數確保檔案不會被分割!
joined_df00.coalesce(1).write.format("csv").option("header","true").save("/home/cloudera/KKBOX/transactions_of_370K/") 